In [4]:
import os
from ultralytics import YOLO
import cv2
import numpy as np

model = YOLO("model.pt")

image_path = "foto.png"
im2 = cv2.imread(image_path)

results = model.predict(source=im2, save=False, save_txt=False)

img_all_blurred = cv2.medianBlur(im2, 25)

mask_img = np.zeros(im2.shape, dtype='uint8')
for box in results[0].boxes:
    x_min, y_min, x_max, y_max = map(int, box.xyxy.tolist()[0])
    w, h = x_max - x_min, y_max - y_min
    ellipse_center = ((x_min + x_max) // 2, (y_min + y_max) // 2)
    axes = (w // 2, h // 2)
    cv2.ellipse(mask_img, ellipse_center, axes, 0, 0, 360, (255, 255, 255), -1)

im2 = np.where(mask_img > 0, img_all_blurred, im2)

base_name, ext = os.path.splitext(image_path)
output_path = f"{base_name}_blurred{ext}"

cv2.imwrite(output_path, im2)


0: 384x640 5 FACEs, 12.1ms
Speed: 7.4ms preprocess, 12.1ms inference, 6.8ms postprocess per image at shape (1, 3, 384, 640)


True

In [3]:
import os
from ultralytics import YOLO
import cv2
import numpy as np

model = YOLO("model.pt")

image_path = "foto.png"
im2 = cv2.imread(image_path)

results = model.predict(source=im2, save=False, save_txt=False)

def pixelate_region(image, x_min, y_min, x_max, y_max, pixel_size=10):
    # Crop the region of interest
    roi = image[y_min:y_max, x_min:x_max]
    
    # Resize to a smaller size
    temp = cv2.resize(roi, (pixel_size, pixel_size), interpolation=cv2.INTER_LINEAR)
    
    # Resize back to the original size
    pixelated = cv2.resize(temp, (x_max - x_min, y_max - y_min), interpolation=cv2.INTER_NEAREST)
    
    # Replace the region with the pixelated version
    image[y_min:y_max, x_min:x_max] = pixelated
    return image

# Create an empty mask with the same size as the image
mask_img = np.zeros(im2.shape, dtype='uint8')

for box in results[0].boxes:
    x_min, y_min, x_max, y_max = map(int, box.xyxy.tolist()[0])
    im2 = pixelate_region(im2, x_min, y_min, x_max, y_max, pixel_size=10)

base_name, ext = os.path.splitext(image_path)
output_path = f"{base_name}_pixelated{ext}"

cv2.imwrite(output_path, im2)
# 


0: 384x640 5 FACEs, 24.6ms
Speed: 4.8ms preprocess, 24.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)


True

In [7]:
import os
import cv2
import numpy as np
from ultralytics import YOLO
import tensorflow as tf

yolo_model = YOLO("model.pt")
print("Modelo YOLO cargado correctamente.")

classification_model = tf.keras.models.load_model("model_lr_0.0001_batch_16_batchnorm_True.h5")
print("Modelo de clasificación cargado correctamente.")

img_height = 200
img_width = 200

def preprocess_image(img, save_path=None):
    img_resized = cv2.resize(img, (img_width, img_height))
    img_normalized = img_resized.astype('float32') / 255.0
    img_expanded = np.expand_dims(img_normalized, axis=0)
    
    if save_path:
        cv2.imwrite(save_path, (img_normalized * 255).astype('uint8'))
    
    return img_expanded

image_path = "foto.png"
im2 = cv2.imread(image_path)
if im2 is not None:
    print("Imagen cargada correctamente.")
else:
    print("Error al cargar la imagen.")

base_name, ext = os.path.splitext(image_path)
preprocessed_dir = f"{base_name}_preprocessed"
os.makedirs(preprocessed_dir, exist_ok=True)

results = yolo_model.predict(source=im2, save=False, save_txt=False)
print(f"Detecciones YOLO: {len(results[0].boxes)} caras detectadas.")

img_all_blurred = cv2.medianBlur(im2, 25)

# Crear una máscara para las áreas que necesitan desenfoque
mask_img = np.zeros(im2.shape, dtype='uint8')

for i, box in enumerate(results[0].boxes):
    x_min, y_min, x_max, y_max = map(int, box.xyxy.tolist()[0])
    face_img = im2[y_min:y_max, x_min:x_max]
    
    preprocessed_face_path = os.path.join(preprocessed_dir, f"preprocessed_face_{i}.png")
    preprocessed_face = preprocess_image(face_img, save_path=preprocessed_face_path)
    print(f"Preprocessed face {i} shape: {preprocessed_face.shape}")
    
    prediction = classification_model.predict(preprocessed_face)
    is_minor = prediction[0][0] > 0.4 
    
    print(f"Predicción: {prediction[0][0]}, Es menor: {is_minor}")
    
    if is_minor:
        w, h = x_max - x_min, y_max - y_min
        ellipse_center = ((x_min + x_max) // 2, (y_min + y_max) // 2)
        axes = (w // 2, h // 2)
        cv2.ellipse(mask_img, ellipse_center, axes, 0, 0, 360, (255, 255, 255), -1)

im2 = np.where(mask_img > 0, img_all_blurred, im2)

output_path = f"{base_name}_blurred{ext}"

cv2.imwrite(output_path, im2)
print(f"Imagen guardada en {output_path}")

Modelo YOLO cargado correctamente.
Modelo de clasificación cargado correctamente.
Imagen cargada correctamente.

0: 384x640 5 FACEs, 13.5ms
Speed: 0.0ms preprocess, 13.5ms inference, 15.6ms postprocess per image at shape (1, 3, 384, 640)
Detecciones YOLO: 5 caras detectadas.
Preprocessed face 0 shape: (1, 200, 200, 3)
1/1 [==============================] - 0s 449ms/step
Predicción: 0.0817958265542984, Es menor: False
Preprocessed face 1 shape: (1, 200, 200, 3)
1/1 [==============================] - 0s 42ms/step
Predicción: 0.13366596400737762, Es menor: False
Preprocessed face 2 shape: (1, 200, 200, 3)
1/1 [==============================] - 0s 41ms/step
Predicción: 0.041347015649080276, Es menor: False
Preprocessed face 3 shape: (1, 200, 200, 3)
1/1 [==============================] - 0s 43ms/step
Predicción: 0.02121875248849392, Es menor: False
Preprocessed face 4 shape: (1, 200, 200, 3)
1/1 [==============================] - 0s 40ms/step
Predicción: 0.027821630239486694, Es menor: Fal

In [10]:
import os
import cv2
import numpy as np
from ultralytics import YOLO
import tensorflow as tf

# Cargar modelo YOLO
yolo_model = YOLO("model.pt")
print("Modelo YOLO cargado correctamente.")

# Lista de modelos de clasificación
model_names = [
    "model_lr_0.01_batch_16_batchnorm_True",
    "model_lr_0.001_batch_16_batchnorm_True",
    "model_lr_0.0005_batch_16_batchnorm_True",
    "model_lr_0.0001_batch_16_batchnorm_True",
    "model_lr_0.01_batch_32_batchnorm_True",
    "model_lr_0.001_batch_32_batchnorm_True",
    "model_lr_0.0005_batch_32_batchnorm_True",
    "model_lr_0.0001_batch_32_batchnorm_True",
    "model_lr_0.01_batch_64_batchnorm_True",
    "model_lr_0.001_batch_64_batchnorm_True",
    "model_lr_0.0005_batch_64_batchnorm_True",
    "model_lr_0.0001_batch_64_batchnorm_True",
    "model_lr_0.01_batch_16_batchnorm_False",
    "model_lr_0.001_batch_16_batchnorm_False",
    "model_lr_0.0005_batch_16_batchnorm_False",
    "model_lr_0.0001_batch_16_batchnorm_False",
    "model_lr_0.01_batch_32_batchnorm_False",
    "model_lr_0.001_batch_32_batchnorm_False",
    "model_lr_0.0005_batch_32_batchnorm_False",
    "model_lr_0.0001_batch_32_batchnorm_False",
    "model_lr_0.01_batch_64_batchnorm_False",
    "model_lr_0.001_batch_64_batchnorm_False",
    "model_lr_0.0005_batch_64_batchnorm_False",
    "model_lr_0.0001_batch_64_batchnorm_False"
]

# Configuración de la imagen
img_height = 200
img_width = 200

def preprocess_image(img, save_path=None):
    img_resized = cv2.resize(img, (img_width, img_height))
    img_normalized = img_resized.astype('float32') / 255.0
    img_expanded = np.expand_dims(img_normalized, axis=0)
    
    if save_path:
        cv2.imwrite(save_path, (img_normalized * 255).astype('uint8'))
    
    return img_expanded

image_path = "foto3.png"
im2 = cv2.imread(image_path)
if im2 is not None:
    print("Imagen cargada correctamente.")
else:
    print("Error al cargar la imagen.")

base_name, ext = os.path.splitext(image_path)
preprocessed_dir = f"{base_name}_preprocessed"
output_dir = f"{base_name}_outputs"
os.makedirs(preprocessed_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

results = yolo_model.predict(source=im2, save=False, save_txt=False)
print(f"Detecciones YOLO: {len(results[0].boxes)} caras detectadas.")

# Iterar sobre todos los modelos y realizar predicciones
for model_name in model_names:
    model_path = f"{model_name}.h5"
    
    if os.path.exists(model_path):
        print(f"Cargando {model_path}...")
        classification_model = tf.keras.models.load_model(model_path)
        print(f"Modelo {model_name} cargado correctamente.")

        img_all_blurred = cv2.medianBlur(im2, 25)
        mask_img = np.zeros(im2.shape, dtype='uint8')

        for i, box in enumerate(results[0].boxes):
            x_min, y_min, x_max, y_max = map(int, box.xyxy.tolist()[0])
            face_img = im2[y_min:y_max, x_min:x_max]
            
            preprocessed_face_path = os.path.join(preprocessed_dir, f"preprocessed_face_{i}.png")
            preprocessed_face = preprocess_image(face_img, save_path=preprocessed_face_path)
            print(f"Preprocessed face {i} shape: {preprocessed_face.shape}")
            
            prediction = classification_model.predict(preprocessed_face)
            is_minor = prediction[0][0] > 0.4 
            
            print(f"Predicción: {prediction[0][0]}, Es menor: {is_minor}")
            
            if is_minor:
                w, h = x_max - x_min, y_max - y_min
                ellipse_center = ((x_min + x_max) // 2, (y_min + y_max) // 2)
                axes = (w // 2, h // 2)
                cv2.ellipse(mask_img, ellipse_center, axes, 0, 0, 360, (255, 255, 255), -1)

        final_image = np.where(mask_img > 0, img_all_blurred, im2)
        output_path = os.path.join(output_dir, f"{base_name}_{model_name}_blurred{ext}")

        cv2.imwrite(output_path, final_image)
        print(f"Imagen guardada en {output_path}")


Modelo YOLO cargado correctamente.
Imagen cargada correctamente.

0: 448x640 8 FACEs, 154.0ms
Speed: 15.6ms preprocess, 154.0ms inference, 0.0ms postprocess per image at shape (1, 3, 448, 640)
Detecciones YOLO: 8 caras detectadas.
Cargando model_lr_0.01_batch_16_batchnorm_True.h5...
Modelo model_lr_0.01_batch_16_batchnorm_True cargado correctamente.
Preprocessed face 0 shape: (1, 200, 200, 3)
1/1 [==============================] - 0s 354ms/step
Predicción: 0.22842366993427277, Es menor: False
Preprocessed face 1 shape: (1, 200, 200, 3)
1/1 [==============================] - 0s 43ms/step
Predicción: 0.3182843327522278, Es menor: False
Preprocessed face 2 shape: (1, 200, 200, 3)
1/1 [==============================] - 0s 42ms/step
Predicción: 0.30062946677207947, Es menor: False
Preprocessed face 3 shape: (1, 200, 200, 3)
1/1 [==============================] - 0s 36ms/step
Predicción: 0.18349306285381317, Es menor: False
Preprocessed face 4 shape: (1, 200, 200, 3)
1/1 [===================